In [1]:
from functions import load_data, epoching , bridged_channels, segment_stimRt
import sys
import os
import numpy as np
import mne
import pickle
from config import data_dir

subject_id = 317

# load the selected bad channels, trials and components
with open(os.path.join( data_dir, 'BadTrialsChannel_manualDetected.pkl'), "rb") as f:
    all_bads = pickle.load(f)
bads_channel= all_bads[subject_id]['channel_names']
bad_trials= all_bads[subject_id]['trial_numbers']
noisy_components = all_bads[subject_id]['noisy_components']

# load raw data
raw = load_data(subject_id)

# Get bridged channels
epochs_unicity = epoching(subject_id, raw, stim  = 'unicity_point' , tmin = -0.5, tmax = 0.2) # a random epoch just for computing the bridged channels
bridged_idx, ed_matrix, bridged_ch_names = bridged_channels(epochs_unicity,   lm_cutoff = 5, epoch_threshold=0.5)

# 1. remove noisy channels
raw.info['bads'] = bads_channel

# 2. Filter the data
raw.notch_filter([50,100], fir_design='firwin', skip_by_annotation='edge')
raw.filter(l_freq=1, h_freq= 30)
# 3. epoch to definition onset ( also you can epoch to unicity point)
epochs_defOnset = epoching(subject_id, raw, stim='onset_definition', tmin=-0.4, tmax=2, baseline=(None))

# 4. remove noisy components
path_ic = os.path.join(data_dir, f'S{subject_id}_ica_infomax.fif')
ica = mne.preprocessing.read_ica(path_ic)
ica.exclude = noisy_components
ica.apply(epochs_defOnset, exclude=noisy_components)

# 5. baseline correction after ICA removal
epochs_defOnset.apply_baseline(baseline=(None, 0))  # baseline correction

# 6. interpolate bridged channels and bad channels
epochs_defOnset = mne.preprocessing.interpolate_bridged_electrodes(epochs_defOnset, bridged_idx, bad_limit=4) 
epochs_defOnset = epochs_defOnset.interpolate_bads()  # interpolate bad channels
# # 7. re-reference to average
# epochs_defOnset.set_eeg_reference(ref_channels='average')  # re-reference to average

# fig = epochs_defOnset.average().plot_joint( title='ERP after ica removal', ts_args=dict(time_unit='s'), topomap_args=dict(time_unit='s'), show = False)

Extracting EDF parameters from c:\Users\gmoha\Downloads\git\EEG_preprocess\Data\A_317_Def_PZ.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 453119  =      0.000 ...   884.998 secs...
Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
108 events found on stim channel Status
Event IDs: [1 2]
Not setting metadata
108 matching events found


c:\Users\gmoha\Downloads\git\EEG_preprocess\functions.py:53: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  xyz_data = pd.read_csv(ch_location_path, delim_whitespace=True, skiprows=1, header=None)


Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 108 events and 359 original time points ...
0 bad epochs dropped


c:\Users\gmoha\Downloads\git\EEG_preprocess\functions.py:87: RuntimeWarning: filter_length (3381) is longer than the signal (359), distortion is likely. Reduce filter length or filter a longer signal.
  bridged_idx, ed_matrix  = compute_bridged_electrodes( epochs, lm_cutoff = lm_cutoff, epoch_threshold= epoch_threshold)


Local minimum 0.8920519772935335 found
Bridge detected between D6 and D7
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 s)



...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (3.303 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.9s


Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
108 events found on stim channel Status
Event IDs: [1 2]
Not setting metadata
108 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 108 events and 1230 original time points ...
0 bad epochs dropped
Reading c:\Users\gmoha\Downloads\git\EEG_preprocess\Data\S317_ica_infomax.fif ...
Now restoring ICA solution ...
Ready.
Applying ICA to Epochs instance
    Transforming to ICA space (125 components)


C:\Users\gmoha\AppData\Local\Temp\ipykernel_824\825972434.py:36: RuntimeWarning: This filename (c:\Users\gmoha\Downloads\git\EEG_preprocess\Data\S317_ica_infomax.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica = mne.preprocessing.read_ica(path_ic)


    Zeroing out 1 ICA component
    Projecting back using 125 PCA components
Applying baseline correction (mode: mean)
Not setting metadata
108 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 127 sensor positions
Interpolating 2 sensors
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 94.6 mm
Computing interpolation matrix from 125 sensor positions
Interpolating 3 sensors


In [ ]:
import autoreject